In [1]:
import pygame
import random

# Pygame initialization
pygame.init()

# Constants
GRID_SIZE = 60  # Size of each grid square
GRID_COLUMNS = 3  # Number of columns in the grid
GRID_ROWS = 2  # Number of rows in the grid
SCREEN_WIDTH = 300  # Window width
SCREEN_HEIGHT = 350  # Window height
RED_SQUARE_SIZE = 20  # Size of the red squares (acid level squares)
ROBOT_SIZE = GRID_SIZE - 20  # Size of the robot
ROBOT_COLOR = (0, 200, 0)  # Green color for the robot
RED_COLOR = (255, 0, 0)  # Red color for high acid squares

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BLUE = (100, 100, 255)

# Initialize screen
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Acid Level Detection Robot")

# Offsets to center the grid
GRID_OFFSET_X = (SCREEN_WIDTH - (GRID_SIZE * GRID_COLUMNS)) // 2
GRID_OFFSET_Y = (SCREEN_HEIGHT - (GRID_SIZE * GRID_ROWS + 50)) // 2 -30

# Possible entrance
entrances = [(2, 0), (1, 0), (0, 0), (0, 1), (1, 1), (2, 1)]

def path_selection(entrance):
    if entrance == (2,0):
        return [(3, 0), (2, 0), (1, 0), (0, 0), (0, 1), (1, 1), (2, 1), (3, 1)]
    elif entrance == (1, 0):
        return [(1, -1), (1, 0), (0, 0), (0, 1), (1, 1), (2, 1), (2, 0), (3,0)]
    elif entrance == (0,0):
        return [(0, -1), (0, 0), (0, 1), (1, 1), (2, 1), (2, 0), (1, 0), (1,-1)]
    elif entrance == (0,1):
        return [(0,2), (0,1), (0,0), (1,0), (2,0), (2,1), (1,1), (1,2)]
    elif entrance == (1,1):
        return [(1,2), (1,1), (0,1), (0,0), (1,0), (2,0), (2,1), (3,1)]
    elif entrance == (2,1):
        return [(3, 1), (2, 1), (1, 1), (0, 1), (0, 0), (1, 0), (2, 0), (3, 0)]

current_step = 0

# Generate random acid level squares (randomly place 2 acid squares in the grid)
acid_squares = set(random.sample([(x, y) for x in range(GRID_COLUMNS) for y in range(GRID_ROWS)], 2))

# Track detected acid squares
detected_acid_squares = []

# Font for displaying text
font = pygame.font.Font(None, 18)

# Function to draw the grid
def draw_grid():
    for row in range(GRID_ROWS):
        for col in range(GRID_COLUMNS):
            x = GRID_OFFSET_X + col * GRID_SIZE
            y = GRID_OFFSET_Y + row * GRID_SIZE
            pygame.draw.rect(screen, WHITE, (x, y, GRID_SIZE, GRID_SIZE), 2)
            # Draw high acid squares (only display acid squares on the grid, not based on robot knowledge)
            if (col, row) in acid_squares:
                pygame.draw.rect(
                    screen,
                    RED_COLOR,
                    (x + (GRID_SIZE - RED_SQUARE_SIZE) // 2, y + (GRID_SIZE - RED_SQUARE_SIZE) // 2, RED_SQUARE_SIZE, RED_SQUARE_SIZE),
                )

# Function to draw the robot
def draw_robot():
    x = GRID_OFFSET_X + robot_x
    y = GRID_OFFSET_Y + robot_y
    pygame.draw.rect(screen, ROBOT_COLOR, (x + 10, y + 10, ROBOT_SIZE, ROBOT_SIZE))

# Function to update the robot's status
def update_status():
    status_text = font.render(f"Status: {status}", True, BLUE)
    count_text = font.render(f"High Acid Squares: {acid_count}", True, BLUE)
    position_text = font.render(f"Current Position: Square {current_step+1}", True, BLUE)
    detected_text = font.render(f"Detected Squares: {detected_acid_squares}", True, BLUE)
    screen.blit(status_text, (10, SCREEN_HEIGHT - 110))
    screen.blit(count_text, (10, SCREEN_HEIGHT - 90))
    screen.blit(position_text, (10, SCREEN_HEIGHT - 70))
    screen.blit(detected_text, (10, SCREEN_HEIGHT - 50))

def draw_menu():
    screen.fill(BLACK)
    font = pygame.font.Font(None, 20)
    
    # Draw "Fixed Entrance" button
    fixed_button = pygame.Rect(75, 100, 150, 50)
    pygame.draw.rect(screen, BLUE, fixed_button)
    
    # Calculate the position to center the text for Fixed Entrance
    fixed_text = font.render("Fixed Entrance", True, WHITE)
    fixed_text_rect = fixed_text.get_rect(center=fixed_button.center)
    screen.blit(fixed_text, fixed_text_rect)
    
    # Draw "Customised Entrance" button
    customised_button = pygame.Rect(75, 180, 150, 50)
    pygame.draw.rect(screen, BLUE, customised_button)
    
    # Calculate the position to center the text for Customised Entrance
    customised_text = font.render("Customised Entrance", True, WHITE)
    customised_text_rect = customised_text.get_rect(center=customised_button.center)
    screen.blit(customised_text, customised_text_rect)

    pygame.display.flip()

def menu_interaction():
    running = True
    while running:
        draw_menu()  # Draw menu screen
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                quit()
            if event.type == pygame.MOUSEBUTTONDOWN:
                mouse_pos = pygame.mouse.get_pos()
                
                # Check if Fixed Entrance button is clicked
                if pygame.Rect(75, 100, 150, 50).collidepoint(mouse_pos):
                    return "fixed"
                
                # Check if Customised Entrance button is clicked
                if pygame.Rect(75, 180, 150, 50).collidepoint(mouse_pos):
                    return "customized"

# Customised entrance: let the user select a starting position
def customised_entrance():
    running = True
    selected_entrance = None
    while running:
        screen.fill(BLACK)
        draw_grid()  # Draw grid

        # Wait for mouse click to select entrance
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                quit()
            if event.type == pygame.MOUSEBUTTONDOWN:
                mouse_pos = pygame.mouse.get_pos()
                # Check for which entrance was clicked
                for entrance in entrances:
                    x = GRID_OFFSET_X + entrance[0] * GRID_SIZE
                    y = GRID_OFFSET_Y + entrance[1] * GRID_SIZE
                    if pygame.Rect(x, y, GRID_SIZE, GRID_SIZE).collidepoint(mouse_pos):
                        selected_entrance = entrance
                        running = False
                        break

        pygame.display.flip()
    return selected_entrance

def main():
    global robot_x, robot_y, status, acid_count, current_step, acid_status

    entrance_choice = menu_interaction()
    
    if entrance_choice == "fixed":
        entrance = (2, 0)  # Predefined entrance
    elif entrance_choice == "customized":
        entrance = customised_entrance()  # Let user choose entrance

    path = path_selection(entrance)
    # Robot and Grid Positioning
    robot_x, robot_y = path[0][0] * GRID_SIZE, path[0][1] * GRID_SIZE
    acid_count = 0
    status = "Stop"  # Initial status before robot enters grid

    clock = pygame.time.Clock()
    running = True

    # Timer to track detection duration
    detection_start_time = None

    while running:
        screen.fill(BLACK)  # Clear the screen before drawing
        draw_grid()  # Draw grid and acid squares
        draw_robot()  # Draw robot on the grid
        update_status()  # Update and display the current status

        # Event handling
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        # Get the current position of the robot
        current_pos = path[current_step]

        if current_pos == path[0]:  # Initial position
            status = "Stop"
            if current_step < len(path) - 1:  # Prepare to move to the next position
                current_step += 1
                robot_x = path[current_step][0] * GRID_SIZE
                robot_y = path[current_step][1] * GRID_SIZE
                status = "Moving"

        elif current_pos == path[-1]:  # Final position
            status = "Stopped at exit"
            screen.fill(BLACK)  # Clear the screen before drawing
            draw_grid()  # Draw grid and acid squares
            draw_robot()
            update_status()
            running = False  # End of movement

        else:
            # Handle detection logic
            if detection_start_time is None:  # Start detecting when reaching a new square
                detection_start_time = pygame.time.get_ticks()
                status = "Detecting"

            if pygame.time.get_ticks() - detection_start_time < 1200:  # Stay in Detecting for 1.5 seconds
                pass  # Robot is still detecting
            else:  # Detection complete
                detection_start_time = None  # Reset detection timer

                if current_pos in acid_squares:  # Check if current square is a high acid square
                    status = "High Acid level detected"
                    acid_count += 1
                    detected_acid_squares.append(current_pos)
                    screen.fill(BLACK)  
                    draw_grid()  
                    draw_robot()
                    update_status()

                # Prepare to move to the next square
                if current_step < len(path) - 1:
                    current_step += 1
                    robot_x = path[current_step][0] * GRID_SIZE
                    robot_y = path[current_step][1] * GRID_SIZE
                    status = "Exploring"

        pygame.display.flip()  
        clock.tick(1) 
    
    def generate_report():
        print("\nFinal Report:")
        print("Detected High Acid Squares:", detected_acid_squares)
        print("Total High Acid Squares Detected:", acid_count)
        print("Robot Navigation Path:", path)

    generate_report()  # Generate and print final report
    pygame.quit()  # Quit pygame

if __name__ == "__main__":
    main()

pygame 2.6.1 (SDL 2.28.4, Python 3.10.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


2024-12-22 15:26:11.625 python[28961:6749469] +[IMKClient subclass]: chose IMKClient_Modern
2024-12-22 15:26:11.625 python[28961:6749469] +[IMKInputSession subclass]: chose IMKInputSession_Modern



Final Report:
Detected High Acid Squares: [(0, 1), (2, 1)]
Total High Acid Squares Detected: 2
Robot Navigation Path: [(0, 2), (0, 1), (0, 0), (1, 0), (2, 0), (2, 1), (1, 1), (1, 2)]
